# LLaDA日本語Alpaca学習 - 最小構成版

## 概要
LoRAを使用してLLaDAモデルを日本語Alpacaデータセットで効率的に学習します。

## 特徴
- ✅ LoRA（パラメータ効率的学習）
- ✅ 混合精度学習
- ✅ 日本語Alpaca指示応答データ
- ✅ メモリ効率最適化

## 必要環境
- GPU: 8GB以上推奨
- Python 3.8+
- CUDA対応

In [ ]:
# 1. 環境設定とライブラリインストール
import sys

# Google Colab対応
if 'google.colab' in sys.modules:
    # 互換性のあるバージョンを明示的にインストール
    print("📦 Installing compatible versions...")
    !pip install transformers==4.49.0 accelerate==0.34.2
    !pip install datasets==2.18.0 peft==0.13.2
    
    # bitsandbyteとtritonの互換バージョンをインストール
    try:
        !pip install bitsandbytes==0.43.1 triton==2.1.0
        print("✅ bitsandbytes and triton installed successfully")
        USE_QUANTIZATION = True
    except:
        print("⚠️ bitsandbytes installation failed, proceeding without quantization")
        USE_QUANTIZATION = False
else:
    USE_QUANTIZATION = True

# 必要ライブラリインポート
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler

from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from peft import LoraConfig, get_peft_model, TaskType
from datasets import load_dataset
import numpy as np
import random
from tqdm import tqdm
import os

# bitsandbyteのインポートを安全に試行
try:
    if USE_QUANTIZATION:
        import bitsandbytes
        print("✅ bitsandbytes imported successfully")
except ImportError:
    print("⚠️ bitsandbytes not available, using standard precision")
    USE_QUANTIZATION = False

# シード設定
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Quantization enabled: {USE_QUANTIZATION}")

In [ ]:
# 2. 設定クラス - 本格学習対応版
class BaseConfig:
    """基本設定"""
    # モデル設定
    MODEL_NAME = 'GSAI-ML/LLaDA-8B-Instruct'
    MASK_ID = 126336
    
    # 保存設定
    OUTPUT_DIR = './llada_japanese_lora'

class TestConfig(BaseConfig):
    """テスト用設定（動作確認）"""
    # データ設定
    DATASET_NAME = 'sample'
    MAX_SAMPLES = 50
    MAX_LENGTH = 128
    
    # 学習設定
    BATCH_SIZE = 1
    GRADIENT_ACCUMULATION = 2
    LEARNING_RATE = 1e-4
    NUM_EPOCHS = 1
    WARMUP_RATIO = 0.1
    
    # LoRA設定
    LORA_R = 8
    LORA_ALPHA = 16
    LORA_DROPOUT = 0.1

class MediumConfig(BaseConfig):
    """中規模学習設定（推奨開始点）"""
    # データ設定
    DATASET_NAME = 'fujiki/japanese_alpaca_data'
    MAX_SAMPLES = 5000      # 5K samples
    MAX_LENGTH = 512        # 長めの文脈
    
    # 学習設定
    BATCH_SIZE = 1
    GRADIENT_ACCUMULATION = 8   # 実効バッチサイズ8
    LEARNING_RATE = 1e-4        # やや高めの学習率
    NUM_EPOCHS = 3              # 十分な学習
    WARMUP_RATIO = 0.1
    
    # LoRA設定
    LORA_R = 32             # 表現力向上
    LORA_ALPHA = 64         # バランス調整
    LORA_DROPOUT = 0.1

class LargeConfig(BaseConfig):
    """大規模学習設定（高性能GPU用）"""
    # データ設定
    DATASET_NAME = 'fujiki/japanese_alpaca_data'
    MAX_SAMPLES = 20000     # 20K samples
    MAX_LENGTH = 768        # 長文対応
    
    # 学習設定
    BATCH_SIZE = 2          # メモリ許可範囲で増加
    GRADIENT_ACCUMULATION = 8   # 実効バッチサイズ16
    LEARNING_RATE = 5e-5        # 安定した学習率
    NUM_EPOCHS = 5              # 深い学習
    WARMUP_RATIO = 0.1
    
    # LoRA設定
    LORA_R = 64             # 高い表現力
    LORA_ALPHA = 128        # 強いLoRA影響
    LORA_DROPOUT = 0.05     # 低いドロップアウト

class ProductionConfig(BaseConfig):
    """本番品質設定（最高品質）"""
    # データ設定
    DATASET_NAME = 'fujiki/japanese_alpaca_data'
    MAX_SAMPLES = 52000     # 全データ活用
    MAX_LENGTH = 1024       # 最長文脈
    
    # 学習設定
    BATCH_SIZE = 1          # 安全なバッチサイズ
    GRADIENT_ACCUMULATION = 16  # 大きな実効バッチサイズ
    LEARNING_RATE = 3e-5        # 慎重な学習率
    NUM_EPOCHS = 8              # 徹底的な学習
    WARMUP_RATIO = 0.1
    
    # LoRA設定
    LORA_R = 128            # 最高の表現力
    LORA_ALPHA = 256        # 最強のLoRA影響
    LORA_DROPOUT = 0.05

# 設定選択（ここを変更して学習レベルを選択）
TRAINING_LEVEL = "medium"  # "test", "medium", "large", "production"

config_map = {
    "test": TestConfig(),
    "medium": MediumConfig(),
    "large": LargeConfig(),
    "production": ProductionConfig()
}

config = config_map[TRAINING_LEVEL]

# 設定情報表示
print(f"=== {TRAINING_LEVEL.upper()} 学習設定 ===")
print(f"データセット: {config.DATASET_NAME}")
print(f"サンプル数: {config.MAX_SAMPLES:,}")
print(f"最大長: {config.MAX_LENGTH}")
print(f"バッチサイズ: {config.BATCH_SIZE}")
print(f"実効バッチサイズ: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION}")
print(f"学習率: {config.LEARNING_RATE}")
print(f"エポック数: {config.NUM_EPOCHS}")
print(f"LoRAランク: {config.LORA_R}")

# メモリ使用量予測
def estimate_memory_usage(config):
    """メモリ使用量の概算"""
    base_memory = 16  # ベースモデルのメモリ（GB）
    sequence_factor = config.MAX_LENGTH / 256  # シーケンス長による係数
    batch_factor = config.BATCH_SIZE
    lora_factor = config.LORA_R / 16  # LoRAランクによる係数
    
    estimated = base_memory * sequence_factor * batch_factor * (1 + lora_factor * 0.1)
    return estimated

estimated_memory = estimate_memory_usage(config)
print(f"\n💾 推定メモリ使用量: {estimated_memory:.1f}GB")

if estimated_memory > 40:
    print("⚠️ 高メモリ使用量！より小さな設定を検討してください")
elif estimated_memory > 24:
    print("⚠️ 高性能GPU推奨（A100など）")
else:
    print("✅ 一般的なGPUで実行可能")

print(f"\n🎯 学習段階の推奨順序:")
print(f"1. test → 動作確認（数分）")
print(f"2. medium → 実用的学習（1-2時間）")
print(f"3. large → 高品質学習（4-8時間）") 
print(f"4. production → 最高品質（12-24時間）")

In [ ]:
# 3. データセット準備
class AlpacaDataset(Dataset):
    def __init__(self, texts, tokenizer):
        self.texts = texts
        self.tokenizer = tokenizer
        self.vocab_size = len(tokenizer)
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        
        # トークン化
        encoding = self.tokenizer(
            text,
            max_length=config.MAX_LENGTH,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)
        
        # 安全性チェック: すべてのトークンIDが語彙サイズ内にあることを確認
        input_ids = torch.clamp(input_ids, 0, self.vocab_size - 1)
        
        # ランダムマスキング（安全版）
        valid_positions = (attention_mask == 1).nonzero(as_tuple=True)[0]
        if len(valid_positions) > 0:
            mask_ratio = random.uniform(0.2, 0.7)
            num_mask = max(1, int(len(valid_positions) * mask_ratio))
            
            # インデックス範囲の安全性チェック
            valid_positions = valid_positions[valid_positions < len(input_ids)]
            if len(valid_positions) > 0:
                num_mask = min(num_mask, len(valid_positions))
                mask_indices = torch.randperm(len(valid_positions))[:num_mask]
                mask_positions = valid_positions[mask_indices]
                
                masked_input_ids = input_ids.clone()
                # MASK_IDも語彙サイズ内であることを確認
                safe_mask_id = min(config.MASK_ID, self.vocab_size - 1)
                masked_input_ids[mask_positions] = safe_mask_id
                
                mask_bool = torch.zeros_like(input_ids, dtype=torch.bool)
                mask_bool[mask_positions] = True
            else:
                masked_input_ids = input_ids.clone()
                mask_bool = torch.zeros_like(input_ids, dtype=torch.bool)
        else:
            masked_input_ids = input_ids.clone()
            mask_bool = torch.zeros_like(input_ids, dtype=torch.bool)
        
        return {
            'input_ids': masked_input_ids,
            'attention_mask': attention_mask,
            'labels': input_ids,
            'mask_positions': mask_bool
        }

def load_alpaca_data():
    """Alpacaデータ読み込み"""
    if config.DATASET_NAME == 'sample':
        # サンプルデータ
        sample_data = [
            "日本の首都について説明してください。\n\n日本の首都は東京です。",
            "健康的な食事のアドバイスをしてください。\n\nバランスの取れた食事が重要です。",
            "プログラミング初心者におすすめの言語を教えてください。\n\nPythonがおすすめです。",
        ]
        return sample_data * (config.MAX_SAMPLES // len(sample_data))
    else:
        # 実データ読み込み
        try:
            dataset = load_dataset(config.DATASET_NAME, split=f'train[:{config.MAX_SAMPLES}]')
            texts = []
            for item in dataset:
                instruction = item.get('instruction', '').strip()
                input_text = item.get('input', '').strip()
                output = item.get('output', '').strip()
                
                if len(instruction) > 5 and len(output) > 10:
                    if input_text:
                        prompt = f"{instruction}\n\n入力: {input_text}"
                    else:
                        prompt = instruction
                    full_text = f"{prompt}\n\n{output}"
                    texts.append(full_text)
            return texts
        except Exception as e:
            print(f"Data loading error: {e}, using sample data")
            config.DATASET_NAME = 'sample'  # フォールバックのために設定変更
            return load_alpaca_data()

# データ読み込み
texts = load_alpaca_data()
print(f"Loaded {len(texts)} samples")
print(f"Sample: {texts[0][:100]}...")

In [ ]:
# 4. モデル準備
# トークナイザー読み込み
tokenizer = AutoTokenizer.from_pretrained(config.MODEL_NAME, trust_remote_code=True)

# MASK_IDの安全性チェック
vocab_size = len(tokenizer)
print(f"Vocabulary size: {vocab_size}")
print(f"Original MASK_ID: {config.MASK_ID}")

# MASK_IDが語彙サイズを超えている場合の修正
if config.MASK_ID >= vocab_size:
    # 利用可能なマスクトークンを使用
    if hasattr(tokenizer, 'mask_token_id') and tokenizer.mask_token_id is not None:
        config.MASK_ID = tokenizer.mask_token_id
    elif hasattr(tokenizer, 'unk_token_id') and tokenizer.unk_token_id is not None:
        config.MASK_ID = tokenizer.unk_token_id
    else:
        # 最後の手段として語彙サイズ-1を使用
        config.MASK_ID = vocab_size - 1
    print(f"⚠️ MASK_ID adjusted to: {config.MASK_ID}")
else:
    print(f"✅ MASK_ID is valid: {config.MASK_ID}")

# ベースモデル読み込み（安全なdevice_map設定）
print(f"Loading model: {config.MODEL_NAME}")
try:
    model = AutoModel.from_pretrained(
        config.MODEL_NAME,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        device_map={'': 0}  # より安全なdevice_map設定
    )
    print("✅ Model loaded successfully")
except Exception as e:
    print(f"⚠️ Loading with device_map failed: {e}")
    print("Trying alternative loading method...")
    model = AutoModel.from_pretrained(
        config.MODEL_NAME,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16
    ).to(device)
    print("✅ Model loaded with alternative method")

# LoRA設定適用（量子化対応）
print(f"Setting up LoRA (quantization: {USE_QUANTIZATION})...")

if USE_QUANTIZATION:
    # 量子化ありでのLoRA設定
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=config.LORA_R,
        lora_alpha=config.LORA_ALPHA,
        lora_dropout=config.LORA_DROPOUT,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        bias="none",
        inference_mode=False,
    )
else:
    # 量子化なしでのLoRA設定
    lora_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        r=config.LORA_R,
        lora_alpha=config.LORA_ALPHA,
        lora_dropout=config.LORA_DROPOUT,
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
        bias="none",
        inference_mode=False,
    )

try:
    model = get_peft_model(model, lora_config)
    model.train()
    print("✅ LoRA applied successfully")
except Exception as e:
    print(f"❌ LoRA setup failed: {e}")
    print("Please restart runtime and try again, or use a different configuration")
    raise e

# パラメータ確認
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable_params/1e6:.1f}M ({trainable_params/all_params*100:.2f}%)")

# データセット作成
train_texts = texts[:int(len(texts)*0.9)]
val_texts = texts[int(len(texts)*0.9):]

train_dataset = AlpacaDataset(train_texts, tokenizer)
val_dataset = AlpacaDataset(val_texts, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=config.BATCH_SIZE, shuffle=False)

print(f"Train batches: {len(train_loader)}, Val batches: {len(val_loader)}")

# メモリ使用量確認
if torch.cuda.is_available():
    try:
        memory_allocated = torch.cuda.memory_allocated() / 1e9
        memory_reserved = torch.cuda.memory_reserved() / 1e9
        print(f"GPU Memory - Allocated: {memory_allocated:.1f}GB, Reserved: {memory_reserved:.1f}GB")
    except:
        print("Could not check GPU memory usage")

In [ ]:
# 5. 学習実行
def compute_loss(model, batch):
    """安全な損失計算"""
    try:
        # 全てのテンソルを同じデバイスに移動
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        mask_positions = batch['mask_positions'].to(device)
        
        # デバッグ情報（最初のバッチでのみ表示）
        if not hasattr(compute_loss, 'debug_printed'):
            print(f"Debug - Input shape: {input_ids.shape}")
            print(f"Debug - Max token ID: {input_ids.max().item()}")
            print(f"Debug - Min token ID: {input_ids.min().item()}")
            print(f"Debug - Vocab size: {len(tokenizer)}")
            compute_loss.debug_printed = True
        
        # トークンIDの安全性チェック
        vocab_size = len(tokenizer)
        input_ids = torch.clamp(input_ids, 0, vocab_size - 1)
        labels = torch.clamp(labels, 0, vocab_size - 1)
        
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        
        # マスク位置での損失計算
        masked_logits = logits[mask_positions]
        masked_labels = labels[mask_positions]
        
        if len(masked_labels) == 0:
            return torch.tensor(0.0, device=device, requires_grad=True)
        
        # 安全な損失計算
        loss = F.cross_entropy(masked_logits, masked_labels, ignore_index=-100)
        
        # NaNチェック
        if torch.isnan(loss) or torch.isinf(loss):
            print(f"Warning: Invalid loss detected: {loss.item()}")
            return torch.tensor(0.0, device=device, requires_grad=True)
        
        return loss
        
    except Exception as e:
        print(f"Error in compute_loss: {e}")
        return torch.tensor(0.0, device=device, requires_grad=True)

# オプティマイザー設定
optimizer = AdamW(model.parameters(), lr=config.LEARNING_RATE, weight_decay=0.01)

total_steps = len(train_loader) * config.NUM_EPOCHS
warmup_steps = int(total_steps * config.WARMUP_RATIO)
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_steps)

# 混合精度設定
scaler = GradScaler()

# 学習ループ
print("Starting training...")
model.train()
global_step = 0
best_val_loss = float('inf')

try:
    for epoch in range(config.NUM_EPOCHS):
        print(f"\nEpoch {epoch+1}/{config.NUM_EPOCHS}")
        
        epoch_losses = []
        optimizer.zero_grad()
        
        for batch_idx, batch in enumerate(tqdm(train_loader, desc="Training")):
            try:
                with autocast():
                    loss = compute_loss(model, batch)
                    loss = loss / config.GRADIENT_ACCUMULATION
                
                scaler.scale(loss).backward()
                epoch_losses.append(loss.item() * config.GRADIENT_ACCUMULATION)
                
                if (batch_idx + 1) % config.GRADIENT_ACCUMULATION == 0:
                    scaler.unscale_(optimizer)
                    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                    scaler.step(optimizer)
                    scaler.update()
                    scheduler.step()
                    optimizer.zero_grad()
                    global_step += 1
                    
            except Exception as e:
                print(f"Error in training step {batch_idx}: {e}")
                optimizer.zero_grad()
                continue
        
        # 検証
        model.eval()
        val_losses = []
        with torch.no_grad():
            for batch in val_loader:
                try:
                    with autocast():
                        loss = compute_loss(model, batch)
                    val_losses.append(loss.item())
                except Exception as e:
                    print(f"Error in validation: {e}")
                    continue
        
        if epoch_losses and val_losses:
            train_loss = np.mean(epoch_losses)
            val_loss = np.mean(val_losses)
            
            print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
            
            # ベストモデル保存
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                os.makedirs(config.OUTPUT_DIR, exist_ok=True)
                model.save_pretrained(config.OUTPUT_DIR)
                print(f"Best model saved: {val_loss:.4f}")
        
        model.train()

except Exception as e:
    print(f"Training error: {e}")
    import traceback
    traceback.print_exc()

print(f"\nTraining completed. Best val loss: {best_val_loss:.4f}")

In [ ]:
# 6. テストと保存
def test_generation(model, tokenizer, prompts):
    """安全な生成テスト"""
    model.eval()
    
    with torch.no_grad():
        for prompt in prompts:
            print(f"\nPrompt: {prompt}")
            
            try:
                # チャットテンプレート適用
                messages = [{"role": "user", "content": prompt}]
                formatted = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)
                input_ids = tokenizer(formatted)['input_ids']
                input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)
                
                prompt_length = input_ids.shape[1]
                gen_length = 64
                
                # 安全なマスクID取得
                vocab_size = len(tokenizer)
                safe_mask_id = min(config.MASK_ID, vocab_size - 1)
                
                # マスク生成
                x = torch.full((1, prompt_length + gen_length), safe_mask_id, dtype=torch.long).to(device)
                x[:, :prompt_length] = input_ids.clone()
                
                # 安全性チェック
                x = torch.clamp(x, 0, vocab_size - 1)
                
                # 予測
                outputs = model(x)
                logits = outputs.logits
                predictions = torch.argmax(logits, dim=-1)
                
                # 予測結果も安全性チェック
                predictions = torch.clamp(predictions, 0, vocab_size - 1)
                
                # マスクを予測で置換
                mask_positions = (x == safe_mask_id)
                x[mask_positions] = predictions[mask_positions]
                
                # デコード
                result = tokenizer.decode(x[0, prompt_length:], skip_special_tokens=True)
                print(f"Result: {result}")
                
            except Exception as e:
                print(f"Generation error for prompt '{prompt}': {e}")
                print("Result: [Generation failed]")

# テスト実行
test_prompts = [
    "日本の首都について説明してください。",
    "健康的な食事のアドバイスをしてください。",
    "プログラミング初心者におすすめの言語を教えてください。"
]

print("=== Generation Test ===")
test_generation(model, tokenizer, test_prompts)

# 最終保存
try:
    model.save_pretrained(config.OUTPUT_DIR)
    print(f"\nModel saved to: {config.OUTPUT_DIR}")
except Exception as e:
    print(f"Error saving model: {e}")

# メモリ使用量確認
if torch.cuda.is_available():
    try:
        memory_used = torch.cuda.max_memory_allocated() / 1e9
        print(f"Max GPU memory used: {memory_used:.1f}GB")
        
        # メモリクリーンアップ
        torch.cuda.empty_cache()
    except Exception as e:
        print(f"Error checking memory: {e}")

print("\n=== Training Complete ===")
print("If CUDA errors occurred, try:")
print("1. Restart the runtime")
print("2. Set config.DATASET_NAME = 'sample' for testing")
print("3. Reduce config.MAX_SAMPLES or config.MAX_LENGTH")